In [1]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [2]:
from llm_calc.lib.experiment import  ls_client, make_df
from llm_calc.lib.config import config
from llm_calc.lib.datamodel import Arm, ArmSlug, Model, ModelSlug
from llm_calc.lib.datacore import datacore
from os.path import join as path_join
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

                                                     LLM CALC                                                      

 ⦿ Using Database Available at https://alx.gd/llm-calc-view-database 

In [8]:
filename = path_join(config.RESULTS_DATA_PATH,"dataset_results_most-recent.pkl")
df = pd.read_pickle(filename)

In [9]:
# exclude last runs (not finished before annotation queue was made)
print(df.shape)
to_exclude = ["melted-cage", "efficacious-romaine", "successful-angina", "known-increase", "pastoral-constitution"]
df = df[~df.experiment_name.isin(to_exclude)]
print(df.shape)

(10000, 59)
(7500, 59)


In [10]:
cdf = df[df.was_correct == False].groupby(["arm_slug","calculator_name"],observed=False).count().id.rename("num_error")
cdf[cdf<5]

arm_slug      calculator_name
gpt4_ci       GAD7               1
gpt4_omc      CAPRINI            0
              GAD7               0
              HASBLED            0
              MELDNa             0
              PSIPORT            0
              SOFA               0
              Wells DVT          4
gpt4_rag      GAD7               0
              HASBLED            1
              Wells DVT          3
gpt4_rag_ci   CCI                4
              GAD7               0
              HASBLED            0
              PSIPORT            0
              Wells DVT          2
llama_omc     GAD7               4
              HASBLED            0
              MELDNa             0
              PSIPORT            0
              Wells DVT          3
llama_rag_ci  GAD7               2
Name: num_error, dtype: int64

In [11]:
# want to sample up to five incorrect runs per arm/calculator
# sample function does not allow for sampling above the population size, so will need to need to exclude those groups for now
max_number_to_sample = cdf
max_number_to_sample[max_number_to_sample > 5] = 5
mdf = pd.DataFrame(max_number_to_sample)
sampled_df = pd.DataFrame()
for row in mdf.iterrows():
    arm_slug, calculator_name = row[0]
    num_to_sample = row[1][0]
    sampled_error_runs = df[(df.arm_slug == arm_slug) & (df.calculator_name == calculator_name) & (df.was_correct == False)].sample(n=num_to_sample)
    sampled_df = pd.concat([sampled_df,sampled_error_runs])
    # print(sampled_error_runs)
sampled_df.groupby(["arm_slug"],observed=True).count()

/var/folders/gr/c5ll10l94zz2tvyf6mzbh3l00000gq/T/ipykernel_68166/533921458.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  num_to_sample = row[1][0]


,id,name,start_time,run_type,end_time,extra,error,serialized,child_runs,app_path,...,output_able_to_answer,num_steps,last_tool_call,last_tool_call_name,arm_description,arm_name,calculator_name,model_slug,model_name,arm_tools
arm_slug,,,,,,,,,,,,,,,,,,,,,
gpt4_base,50,50,50,50,50,50,0,0,0,50,...,50,50,0,0,50,50,50,50,50,50
gpt4_ci,46,46,46,46,46,46,0,0,0,46,...,46,46,46,46,46,46,46,46,46,46
gpt4_omc,19,19,19,19,19,19,0,0,0,19,...,19,19,19,19,19,19,19,19,19,19
gpt4_rag,39,39,39,39,39,39,0,0,0,39,...,39,39,0,0,39,39,39,39,39,39
gpt4_rag_ci,31,31,31,31,31,31,0,0,0,31,...,31,31,31,31,31,31,31,31,31,31
llama_base,50,50,50,50,50,50,0,0,0,50,...,44,50,0,0,50,50,50,50,50,50
llama_ci,50,50,50,50,50,50,0,0,0,50,...,50,50,50,50,50,50,50,50,50,50
llama_omc,32,32,32,32,32,32,0,0,0,32,...,32,32,32,32,32,32,32,32,32,32
llama_rag,50,50,50,50,50,50,0,0,0,50,...,44,50,0,0,50,50,50,50,50,50


In [12]:
# ensure no duplicates in the sampled data
print(len(sampled_df))
print(len(sampled_df.id.value_counts()>1))

414
414


In [13]:
sdf = sampled_df
df_subset_1 = sdf.sample(164)
sdf = sdf.drop(df_subset_1.index)
df_subset_2 = sdf.sample(150)
sdf = sdf.drop(df_subset_2.index)
df_subset_3 = sdf.sample(100)
sdf = sdf.drop(df_subset_3.index)

In [16]:
# filename = path_join(config.RESULTS_DATA_PATH, "annotation", "all_tasks_sent.csv")
# sdf.to_csv(filename)
#
# filename = path_join(config.RESULTS_DATA_PATH, "annotation", "subset_1_alex.csv")
# df_subset_1.to_csv(filename)
#
# filename = path_join(config.RESULTS_DATA_PATH, "annotation", "subset_2_simon.csv")
# df_subset_2.to_csv(filename)
#
# filename = path_join(config.RESULTS_DATA_PATH, "annotation", "subset_3_larry.csv")
# df_subset_3.to_csv(filename)


In [9]:
def add_to_queue(queue_name,data):
    # move to annotation queue
    dataset_queues = pd.DataFrame(
        ls_client.list_annotation_queues(name=queue_name)
    )
    q_url = ""
    if len(dataset_queues) > 0:
        dataset_queues_df = make_df(dataset_queues)
        queue = dataset_queues_df.iloc[0]
        queue_id = queue["id"]
        queue_name = queue["name"]
        q_url = f"https://smith.langchain.com/o/80231dbb-1e31-4379-b804-df697b777bc6/annotation-queues/{str(queue_id)}?peekedRun="
        print(f"Queue already exsists: {queue_name} located at: ")
        print(q_url)
    else:
        # make annotation queue
        queue = ls_client.create_annotation_queue(
            name=queue_name, description=f"annotation queue for {queue_name}"
        )
        queue_id = queue.id
        queue_name = queue.name
        q_url = f"https://smith.langchain.com/o/80231dbb-1e31-4379-b804-df697b777bc6/annotation-queues/{str(queue_id)}?peekedRun="
        print(f"Created queue: {queue_name} located at:")
        print(q_url)
        ls_client.add_runs_to_annotation_queue(
            queue_id=queue_id, run_ids=data.id
        )
        print(f"Added {len(data)} runs to queue {queue_name}")
    return q_url

In [10]:
testing = add_to_queue("LLMCalc -- Error Classification Test Queue",df_subset_1) # add runs to annotation queue

Created queue: LLMCalc -- Error Classification Test Queue located at:
https://smith.langchain.com/o/80231dbb-1e31-4379-b804-df697b777bc6/annotation-queues/49fd4e0b-f20d-4100-8b50-648b9915b464?peekedRun=
Added 164 runs to queue LLMCalc -- Error Classification Test Queue


In [11]:
alex = add_to_queue("LLMCalc -- Error Classification Alex Goodell",df_subset_1)

Created queue: LLMCalc -- Error Classification Alex Goodell located at:
https://smith.langchain.com/o/80231dbb-1e31-4379-b804-df697b777bc6/annotation-queues/411ece5f-98b1-4211-81b8-5c2a7dee0aa6?peekedRun=
Added 164 runs to queue LLMCalc -- Error Classification Alex Goodell


In [12]:
simon = add_to_queue("LLMCalc -- Error Classification Simon Chu",df_subset_2)

Created queue: LLMCalc -- Error Classification Simon Chu located at:
https://smith.langchain.com/o/80231dbb-1e31-4379-b804-df697b777bc6/annotation-queues/e7de1584-c983-4f33-86f3-70eca8169028?peekedRun=
Added 150 runs to queue LLMCalc -- Error Classification Simon Chu


In [13]:
larry =add_to_queue("LLMCalc -- Error Classification Larry Chu",df_subset_3)

Created queue: LLMCalc -- Error Classification Larry Chu located at:
https://smith.langchain.com/o/80231dbb-1e31-4379-b804-df697b777bc6/annotation-queues/e949624e-037c-44b3-9a3c-401e8cbdfc9a?peekedRun=
Added 100 runs to queue LLMCalc -- Error Classification Larry Chu


In [14]:
sdf

,id,name,start_time,run_type,end_time,extra,error,serialized,child_runs,app_path,...,output_able_to_answer,num_steps,last_tool_call,last_tool_call_name,arm_description,arm_name,calculator_name,model_slug,model_name,arm_tools


In [15]:
print(f"""
Tesing: {testing}
Alex: {alex}
Simon: {simon}
Larry: {larry}
""")


Tesing: https://smith.langchain.com/o/80231dbb-1e31-4379-b804-df697b777bc6/annotation-queues/49fd4e0b-f20d-4100-8b50-648b9915b464?peekedRun=
Alex: https://smith.langchain.com/o/80231dbb-1e31-4379-b804-df697b777bc6/annotation-queues/411ece5f-98b1-4211-81b8-5c2a7dee0aa6?peekedRun=
Simon: https://smith.langchain.com/o/80231dbb-1e31-4379-b804-df697b777bc6/annotation-queues/e7de1584-c983-4f33-86f3-70eca8169028?peekedRun=
Larry: https://smith.langchain.com/o/80231dbb-1e31-4379-b804-df697b777bc6/annotation-queues/e949624e-037c-44b3-9a3c-401e8cbdfc9a?peekedRun=

